In [2151]:
# Imports
import pandas as pd
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Dict, Iterable, List, Tuple
import csv
import io
import math

In [2152]:
# Fonction de conversion des floats

def fr_float(s: str | None) -> Optional[float]:
    """Convertit une chaîne avec virgule en float (ou None si vide)."""
    if s is None:
        return None
    s = s.strip().strip('"').strip("'")
    if s == "" or s.lower() in {"nan", "null"}:
        return None
    # Remplacer la virgule décimale par un point
    return float(s.replace(",", "."))


>Définition du mur et de la piece

In [2153]:
# Définition de la pièce
@dataclass
class Piece:
    id: str
    niv_sol: float = 0.0
    surface: float = 0.0
    volume: float = 0.0

    @staticmethod
    def from_csv_row(row: dict) -> "Piece":
        """Crée une pièce depuis une ligne CSV (clés: Piece, niv_sol..)."""
        return Piece(
            id=row.get("Piece", "").strip(),
            niv_sol=fr_float(row.get("niv_sol")),
            surface=fr_float(row.get("surface")),
            volume=fr_float(row.get("volume")),
        )

In [2154]:
# Définition du mur
@dataclass
class Mur:
    """Représente un mur avec ses coordonnées/mesures."""
    id: str
    x: Optional[float]
    y: Optional[float]
    z: Optional[float]
    centre_x: Optional[float]
    centre_y: Optional[float]
    centre_z: Optional[float]
    piece: Optional[Piece] = None

    @property
    def niv_sol(self) -> float:
        """Retourne toujours le niveau de la pièce associée."""
        return self.piece.niv_sol if self.piece else 0.0

    @staticmethod
    def from_csv_row(row: dict) -> "Mur":
        """Crée un Mur depuis une ligne CSV (clés: Murs, X, Y, Z)."""
        id=row.get("Murs", "").strip()
        x=fr_float(row.get("X"))
        y=fr_float(row.get("Y"))
        z=fr_float(row.get("Z"))
        centre_x=fr_float(row.get("centre_x"))
        centre_y=fr_float(row.get("centre_y"))
        centre_z=fr_float(row.get("centre_z"))

        if id[-2] == "X" :
            y = y/2
        if id[-2] == "Y" :
            x = x/2
        
        return Mur(
            id,
            x,
            y,
            z,
            centre_x,
            centre_y,
            centre_z,
        )

>Chargement des pieces, murs et frontieres

In [2155]:
# Lire_pieces_csv

def lire_pieces_csv(source: str | io.TextIOBase) -> Dict[str, Piece]:
    """
    Lit pieces.csv (colonnes: Piece, niv_sol) et renvoie {id_piece: Piece}.
    Gère la virgule décimale ("3,5" -> 3.5).
    """
    
    stream = open(source, "r", encoding="utf-8")

    reader = csv.DictReader(stream)
    pieces: Dict[str, Piece] = {}
    for row in reader:
        piece = Piece.from_csv_row(row)
        if not piece.id:
            # Ignore les lignes sans identifiant
            continue
        pieces[piece.id] = piece
    return pieces

In [2156]:
# Lire_murs_csv

def lire_murs_csv(source: str | io.TextIOBase) -> Dict[str, Mur]:
    """
    Lit un CSV et renvoie un dict {id_mur: Mur}.
    Accepte des valeurs X/Y/Z avec virgule comme décimale.
    """
    stream = open(source, "r", encoding="utf-8")

    reader = csv.DictReader(stream)
    murs: Dict[str, Mur] = {}
    for row in reader:
        mur = Mur.from_csv_row(row)
        if not mur.id:
            # Ignore les lignes sans identifiant
            continue
        murs[mur.id] = mur
    return murs

In [2157]:
# Lire_frontieres_csv

def lire_frontieres_csv(source: str | io.TextIOBase) -> List[Tuple[str, str]]:
    """
    Lit un CSV de frontières (colonnes: Mur1, Mur2)
    et renvoie une liste de couples (mur1, mur2) uniques.
    """
    stream = open(source, "r", encoding="utf-8")
    reader = csv.DictReader(stream)
    frontieres_set = set()  # pour éviter les doublons

    for row in reader:
        m1 = row.get("Mur1", "").strip()
        m2 = row.get("Mur2", "").strip()
        if not m1 or not m2:
            continue

        # Tri pour éviter les inversions (("A","B") == ("B","A"))
        pair = tuple(sorted((m1, m2)))
        frontieres_set.add(pair)

    # Ferme le fichier si on l’a ouvert ici
    if isinstance(source, (str, bytes)):
        stream.close()

    # Convertit l’ensemble en liste triée
    return frontieres_set


In [2158]:
# Assigner_pieces_aux_murs

def piece_id_from_mur_id(mur_id: str) -> str:
    """
    Ex: '0AX0'  -> '0A'
        '0BY1'  -> '0B'
        '1ABX0' -> '1AB'
    Règle: on enlève les 2 derniers caractères (X0/X1/Y0/Y1).
    """
    if len(mur_id) < 3:
        raise ValueError(f"id mur trop court: {mur_id}")
    return mur_id[:-2]

def assigner_pieces_aux_murs(murs: Dict[str, Mur], pieces: Dict[str, Piece], strict: bool = False) -> None:
    """
    Pour chaque Mur du dict `murs`, déduit l'id de pièce et assigne `mur.piece`.
    Si `strict=True`, lève si la pièce n'existe pas. Sinon, ignore en avertissant.
    """
    for mur in murs.values():
        pid = piece_id_from_mur_id(mur.id)
        piece = pieces.get(pid)
        if piece is None:
            if strict:
                raise KeyError(f"Pièce '{pid}' introuvable pour le mur {mur.id}")
            # sinon, on laisse mur.piece=None (ou log si tu as un logger)
        else:
            mur.piece = piece

>Trouver la longueur et largeur

In [2159]:
# Frontiere_nord
def frontiere_nord(piece, frontieres):
    target = "0"+piece+"X1"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    return False 

In [2160]:
# Longueur_totale_y
def longueur_totale_y(murs, frontieres):
    l_tot = 0
    piece_select = "DEPART"

    while piece_select != False :
        piece_select = frontiere_nord(piece_select, frontieres)
        if piece_select !=False :
            l_tot += ( murs["0" + piece_select + "X0"].y ) + murs["0" + piece_select + "Y0"].y + ( murs["0" + piece_select + "X1"].y )   #Le mur + l'épaisseur des murs en travers
    return round(l_tot, 3)

In [2161]:
# Frontiere_ouest
def frontiere_ouest(piece, frontieres):
    target = "0"+piece+"Y0"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    return False 

In [2162]:
# Largeur_totale_x
def largeur_totale_x(murs, frontieres):
    l_tot = 0
    piece_select = "DEPART"

    while piece_select != False :
        piece_select = frontiere_ouest(piece_select, frontieres)
        if piece_select !=False :
            l_tot += ( murs["0" + piece_select + "Y0"].x ) + murs["0" + piece_select + "X0"].x + ( murs["0" + piece_select + "Y1"].x )   #Le mur + l'épaisseur des murs en travers
    return round(l_tot, 3) 

>Positionnement des murs

In [2163]:
# Frontiere_est
def frontiere_est(piece, frontieres, num_etage):
    target = str(num_etage) + piece+"Y1"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    raise ValueError(f"Pas trouvé de frontière est pour {target}")

In [2164]:
# Frontiere_sud
def frontiere_sud(piece, frontieres, num_etage):
    target = str(num_etage)+piece+"X0"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[1:-2]
        if mur2 == target :
            return mur1[1:-2]
    raise ValueError(f"Pas trouvé de frontière sud pour {target}")

In [2165]:
# Place_piece
def place_piece(piece, murs, frontieres, largeur_totale_x, longueur_totale_y, num_etage) :
    fsud = frontiere_sud(piece,frontieres, num_etage)
    fest = frontiere_est(piece,frontieres, num_etage)

    max_x = largeur_totale_x/2
    max_y = longueur_totale_y/2

    #Mur du bas X0 : 

    mur = murs[str(num_etage) + piece + "X0"]
    #centre_y
    if(fsud) == "DEPART" :
        centre_y = -1 * max_y + mur.y /2
        mur.centre_y = round(centre_y, 3) 
    else :
        centre_y = murs[str(num_etage) + fsud + "X1"].centre_y + murs[str(num_etage) + fsud + "X1"].y /2 + mur.y /2
                    #  La position du mur de frontiere + la moitié du mur de frontiere + la moitié du nouveau mur (son centre)
        mur.centre_y = round(centre_y, 3) 
    
    #centre_x
    if(fest) == "DEPART" :
        centre_x = max_x - murs[str(num_etage) + piece + "Y1"].x - mur.x /2
        mur.centre_x = round(centre_x, 3) 
    else :
        centre_x = murs[str(num_etage) + fest + "Y0"].centre_x - murs[str(num_etage) + fest + "Y0"].x / 2 - murs[str(num_etage) + piece + "Y1"].x - mur.x/2
        mur.centre_x = round(centre_x, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z

    #Mur du haut X1 : 

    mur = murs[str(num_etage) + piece + "X1"]
    #centre_y
    centre_y =  murs[str(num_etage) + piece + "X0"].centre_y + murs[str(num_etage) + piece + "X0"].y / 2 + murs[str(num_etage) + piece + "Y0"].y + mur.y / 2
    mur.centre_y = round(centre_y, 3) 

    #centre_x
    centre_x = murs[str(num_etage) + piece + "X0"].centre_x
    mur.centre_x = round(centre_x, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z

    #Mur de droite Y1 :
    mur = murs[str(num_etage) + piece + "Y1"]
    #centre_x
    if(fest) == "DEPART" :
        centre_x = max_x - mur.x / 2
        mur.centre_x = round(centre_x, 3) 
    else :
        centre_x = murs[str(num_etage) + fest + "Y0"].centre_x - murs[str(num_etage) + fest + "Y0"].x /2 - mur.x / 2
        mur.centre_x = round(centre_x, 3) 

    #centre_y
    if(fsud) == "DEPART" :
        centre_y = -1 * max_y + murs[str(num_etage) + piece + "X0"].y + mur.y/2
        mur.centre_y = round(centre_y, 3) 
    else :
        centre_y = murs[str(num_etage) + piece + "X0"].centre_y + murs[str(num_etage) + piece + "X0"].y /2 + mur.y/2
        mur.centre_y = round(centre_y, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z

    #Mur de Gauche Y0 :

    mur = murs[str(num_etage) + piece + "Y0"]
    #centre_x
    centre_x = murs[str(num_etage) + piece + "X0"].centre_x - murs[str(num_etage) + piece + "X0"].x / 2 - mur.x /2
    mur.centre_x = round(centre_x, 3) 

    #centre_y
    centre_y = murs[str(num_etage) + piece + "Y1"].centre_y
    mur.centre_y = round(centre_y, 3) 

    #centre_z
    centre_z = mur.niv_sol + mur.z/2
    mur.centre_z = centre_z



>Ajout automatique des sols et plafonds

In [2166]:
# Frontiere_dessous
def frontiere_dessous(piece, frontieres, num_etage):
    target = piece + "Z0"
    for mur1, mur2 in frontieres:
        if mur1 == target :
            return mur2[:-2]
        if mur2 == target :
            return mur1[:-2]
    raise ValueError(f"Pas trouvé de frontière dessous pour {target}")

In [2167]:
# Place_sols_et_plafonds
def place_sols_et_plafonds(liste_de_pieces,murs,pieces, frontieres):
    i_etage = -1
    for etage in liste_de_pieces :
        i_etage+=1
        for piece in etage :
            piece = str(i_etage)+piece
            id_sol= piece+"Z0"
            x_sol=murs[piece+"X0"].x
            y_sol=murs[piece+"Y0"].y
            if piece[0] == "0" :
                z_sol=0.15
            else :
                mur_dessous = frontiere_dessous(piece, frontieres, i_etage) # du type "0A"
                z_sol = (murs[piece+"X0"].centre_z - murs[piece+"X0"].z/2) - (murs[mur_dessous+"X0"].centre_z + murs[mur_dessous+"X0"].z/2)
                
            centre_x_sol= murs[piece+"X0"].centre_x
            centre_y_sol= murs[piece+"Y0"].centre_y
            centre_z_sol= (murs[piece+"X0"].centre_z - murs[piece+"X0"].z/2) - z_sol/2
            piece_sol=pieces[piece]

            mur = Mur(id=id_sol, x=x_sol, y=y_sol, z=z_sol, centre_x=centre_x_sol, centre_y=centre_y_sol, centre_z=centre_z_sol, piece= piece_sol)
            murs[mur.id] = mur

>Ajustement de la longueur des murs pour faire des blocs

In [2168]:
# Ajuste_longueur_mur
def Ajuste_longueur_mur(murs):
    for mur in murs :
        if murs[mur].id[-2] == "X" or murs[mur].id[-2] == "Z":
            murs[mur].x += murs[murs[mur].id[:-2]+"Y0"].x + murs[murs[mur].id[:-2]+"Y1"].x
            murs[mur].centre_x += murs[murs[mur].id[:-2]+"Y0"].x/2 - murs[murs[mur].id[:-2]+"Y1"].x/2

        if murs[mur].id[-2] == "Y" or murs[mur].id[-2] == "Z":
            murs[mur].y += murs[murs[mur].id[:-2]+"X0"].y + murs[murs[mur].id[:-2]+"X1"].y
            murs[mur].centre_y -= murs[murs[mur].id[:-2]+"X0"].y/2 - murs[murs[mur].id[:-2]+"X1"].y/2

        if murs[mur].id[-2] == "X" or murs[mur].id[-2] == "Y" :
            murs[mur].z += murs[murs[mur].id[:-2]+"Z0"].z
            murs[mur].centre_z -= murs[murs[mur].id[:-2]+"Z0"].z/2


>Calcul Superficie et volume

In [2169]:
def ecrire_csv(chemin_csv: str, lignes: list[dict]):
    """
    Écrit un fichier CSV à partir d'une liste de dictionnaires.
    Chaque clé du premier dictionnaire sert d'en-tête.
    """
    if not lignes:
        print("⚠️ Aucune donnée à écrire.")
        return

    with open(chemin_csv, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=lignes[0].keys())
        writer.writeheader()
        writer.writerows(lignes)

    print(f"✅ CSV écrit avec succès : {chemin_csv}")

def ajoute_superficie_volume_pieces(pieces, murs):
    """
    Calcule la superficie et la position centrale de chaque pièce,
    puis écrit un CSV `labels.csv` pour affichage dans Blender.
    """
    data = []
    surface_totale = 0
    for key, piece in pieces.items():
        # Calcule la surface à partir du mur Z0 (exemple)
        mur_z0 = murs.get(key + "Z0")
        mur_x0 = murs.get(key + "X0")

        if not mur_z0 or not mur_x0:
            print(f"⚠️ Murs manquants pour la pièce {key}")
            continue

        surface = (mur_z0.x or 0) * (mur_z0.y or 0)
        piece.surface = surface

        # Coordonnées du centre
        centre_x = mur_z0.centre_x
        centre_y = mur_z0.centre_y
        centre_z = (mur_z0.centre_z or 0) + (mur_x0.z or 0) / 2

        # Ajoute au tableau
        data.append({
            "texte": f"{key} : {round(surface, 2)} m²",
            "centre_x": centre_x,
            "centre_y": centre_y,
            "centre_z": centre_z
        })
        surface_totale += surface


    ecrire_csv("./data/labels.csv", data)
    print(surface_totale)


> Ecriture dans un csv final des murs avec les bonnes poisition 

In [2170]:
# Ecrire_murs_csv
def ecrire_murs_csv(murs: Dict[str, Mur], chemin: str) -> None:
    """
    Exporte le dictionnaire `murs` dans un fichier CSV.
    Colonnes : Murs, X, Y, Z, centre_x, centre_y, centre_z
    Les floats sont formatés avec virgule décimale.
    """
    champs = ["Murs", "X", "Y", "Z", "centre_x", "centre_y", "centre_z"]

    with open(chemin, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=champs)
        writer.writeheader()

        for mur in murs.values():
            # Conversion en dict de base
            d = asdict(mur)
            # Format des nombres avec virgule décimale (ou vide si None)
            def fmt(v):
                if v is None:
                    return ""
                if isinstance(v, (float, int)):
                    return str(v).replace(".", ",")
                return str(v)

            ligne = {
                "Murs": mur.id,
                "X": fmt(mur.x),
                "Y": fmt(mur.y),
                "Z": fmt(mur.z),
                "centre_x": fmt(mur.centre_x),
                "centre_y": fmt(mur.centre_y),
                "centre_z": fmt(mur.centre_z),
            }
            writer.writerow(ligne)

    print(f"✅ Fichier CSV écrit : {chemin}")


In [2171]:
# Main du notebook

liste_des_pieces = [["A","B","C","D","E","F","G"], ["AB","AA","AC","B","C","D","E","FA","FB"], ["AB","AA","AC","B","C","D"]]

murs = lire_murs_csv("./data/murs.csv")
frontieres = lire_frontieres_csv("./data/frontiere.csv")
pieces = lire_pieces_csv("./data/pieces.csv")

assigner_pieces_aux_murs(murs, pieces, strict=True)

long_tot_y = longueur_totale_y(murs,frontieres)
larg_tot=x = largeur_totale_x(murs,frontieres)

num_etage = 0

for etage in liste_des_pieces :
    for piece in etage :
        place_piece(piece, murs, frontieres, larg_tot, long_tot_y, num_etage)
    num_etage += 1

place_sols_et_plafonds(liste_des_pieces,murs,pieces, frontieres)
ajoute_superficie_volume_pieces(pieces, murs)
print(pieces)
#Ajuste_longueur_mur(murs)

✅ CSV écrit avec succès : ./data/labels.csv
472.72
{'0A': Piece(id='0A', niv_sol=-0.4, surface=33.599999999999994, volume=None), '0B': Piece(id='0B', niv_sol=-0.2, surface=18.0, volume=None), '0C': Piece(id='0C', niv_sol=-0.2, surface=30.0, volume=None), '0D': Piece(id='0D', niv_sol=0.0, surface=44.400000000000006, volume=None), '0E': Piece(id='0E', niv_sol=0.0, surface=14.0, volume=None), '0F': Piece(id='0F', niv_sol=0.0, surface=11.2, volume=None), '0G': Piece(id='0G', niv_sol=0.0, surface=22.05, volume=None), '1AA': Piece(id='1AA', niv_sol=2.9, surface=13.049999999999999, volume=None), '1AB': Piece(id='1AB', niv_sol=2.9, surface=17.36, volume=None), '1AC': Piece(id='1AC', niv_sol=2.9, surface=3.19, volume=None), '1B': Piece(id='1B', niv_sol=3.0, surface=18.0, volume=None), '1C': Piece(id='1C', niv_sol=2.85, surface=36.0, volume=None), '1D': Piece(id='1D', niv_sol=2.75, surface=38.400000000000006, volume=None), '1E': Piece(id='1E', niv_sol=3.5, surface=30.749999999999996, volume=None

In [2172]:
# Utilisation de ecrire_murs_csv

ecrire_murs_csv(murs, "./data/murs_export.csv")


✅ Fichier CSV écrit : ./data/murs_export.csv
